### Finetuning a CNN in Pytorch

Here are the steps you need to do to finetune a CNN in PyTorch

1. To finetune a CNN, we first need to select a pre-trained model. Pytorch already has a few pre-trained models available and to access them, you need to first import them:

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

2. Next, we need to create our model. When creating our model we need to freeze all the convolutional layers which we do by their `requires_grad()` attribute to `False`. We also need to add a fully connected layer on top of it which we do use the Sequential API.

In [ ]:
def create_model():
    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False# means freeze convolutional layer   

    num_features=model.fc.in_features#number of features 
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    return model

3. Training our model will go faster if we use the GPU. We first check if GPU is available by checking for `cuda` devices. To use the GPU we need to put the model on the GPU device by using the `to()` method of the model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on Device {device}")

model=create_model()
model=model.to(device)# Activates GPU

4. Finally, we can create our training loop. This loop remains the same as before, except that we need to put our data on the GPU device as well by using the same `to()` method

In [ ]:
def train(model, train_loader, cost, optimizer, epoch):
 model.train()
 for e in range(epoch):
     running_loss=0
     correct=0
     for data, target in train_loader:
         data=data.to(device)
         target=target.to(device)
         optimizer.zero_grad()
         pred = model(data)             #No need to reshape data since CNNs take image inputs
         loss = cost(pred, target)
         running_loss+=loss
         loss.backward()
         optimizer.step()
         pred=pred.argmax(dim=1, keepdim=True)
         correct += pred.eq(target.view_as(pred)).sum().item()
     print(f"Epoch {e}: Loss {running_loss/len(train_loader.dataset)}, \
         Accuracy {100*(correct/len(train_loader.dataset))}%")